In [1]:
from typing import List, Tuple

from morphosamplers.spline import Spline3D
import napari
import networkx as nx
import numpy as np
from scipy.ndimage import map_coordinates


from skeleton import Skeleton3D, merge_nodes
from skeleton import EDGE_COORDINATES_KEY, EDGE_SPLINE_KEY, NODE_COORDINATE_KEY, EDGE_FEATURES_START_NODE_KEY, EDGE_FEATURES_END_NODE_KEY

from skeleton import SkeletonViewer

In [2]:
node_coordinates = np.array(
    [
        [10, 25, 25],
        [20, 25, 25],
        [40, 35, 25],
        [40, 15, 25],
        [40, 30, 30],
        [20, 45, 45],
        [20, 15, 15]
    ]
)

edges = [
    (0, 1),
    (1, 2),
    (1, 3),
    (4, 5),
    (6, 4)
]


skeleton_graph = nx.Graph(edges)

In [3]:
def _paint_images_from_coordinates_list(
    image: np.ndarray,
    coordinates_to_paint: np.ndarray,
    fill_value: float = 1
):
    # must be 3D image and coordinates
    assert image.ndim == 3
    assert coordinates_to_paint.shape[1] == 3
    
    image[
        coordinates_to_paint[:, 0],
        coordinates_to_paint[:, 1],
        coordinates_to_paint[:, 2],
    ] = fill_value
    
    return image


def _get_coordinantes_in_branch(
    start_point: np.ndarray,
    end_point: np.ndarray
    
) -> np.ndarray:
    # we oversample the line to make sure it is continuous
    line_length = np.linalg.norm(end_point - start_point)
    n_skeleton_points = 2 * int(line_length)
    
    coordinates = np.linspace(
        start_point,
        end_point,
        n_skeleton_points
    ).astype(int)
    
    # return the unique coordinates
    unique_coordinates, unique_indices = np.unique(coordinates, axis=0, return_index=True)
    return unique_coordinates[np.argsort(unique_indices)] 



def make_skeleton_image(
    node_coordinates: np.ndarray,
    edge_list: List[Tuple[int, int]],
    image_shape: Tuple[int, int, int]
) -> Tuple[np.ndarray, List[np.ndarray]]:
    
    image = np.zeros(image_shape)
    
    # get the coordinates of the points on the edges
    coordinates = [
        _get_coordinantes_in_branch(
            node_coordinates[start_index],
            node_coordinates[end_index],
        )
        for start_index, end_index in edge_list
    ]
    # paint the edges
    image = _paint_images_from_coordinates_list(
        image=image,
        coordinates_to_paint=np.concatenate(coordinates)
    )
    
    return image, coordinates

In [4]:
skeleton_image, edge_coordinates = make_skeleton_image(
    node_coordinates=node_coordinates,
    edge_list=edges,
    image_shape=(50, 50, 50)
)

In [5]:
edge_properties = {}
for edge, coordinates in zip(edges, edge_coordinates):
    spline = Spline3D(points=coordinates)
    
    edge_properties.update(
        {
            edge: {
                EDGE_COORDINATES_KEY: coordinates,
                EDGE_SPLINE_KEY: spline,
                EDGE_FEATURES_START_NODE_KEY: edge[0],
                EDGE_FEATURES_END_NODE_KEY: edge[1]
            }
        }
    )

nx.set_edge_attributes(skeleton_graph, edge_properties)

In [6]:
nx.set_edge_attributes(
    skeleton_graph,
    {
        (0, 1): {"validated": False}
    }
)

In [7]:
node_properties = {}

for node_index, coordinate in enumerate(node_coordinates):
    node_properties.update(
        {
            node_index: {
                NODE_COORDINATE_KEY: coordinate
            }
        }
    )
nx.set_node_attributes(skeleton_graph, node_properties)

In [8]:
viewer = napari.Viewer()


/Users/kyamauch/mambaforge/envs/morphometrics/lib/python3.9/site-packages/morphometrics_engine/measure.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [9]:
viewer.add_image(skeleton_image, scale=(1, 2, 1))

<Image layer 'skeleton_image' at 0x1619812b0>

In [10]:
skeleton = Skeleton3D.parse(
    graph=skeleton_graph,
    edge_attributes={"validated": True},
    edge_coordinates_key="edge_coordinates",
    node_coordinate_key="node_coordinate",
    scale=(1, 2, 1)
)

# verify that the graph is a copy
assert skeleton.graph is not skeleton_graph


In [11]:
# merge nodes 2 and 4 using the model
skeleton.merge_nodes(node_to_keep=2, node_to_merge=4)

In [12]:
# make the skeleton viewer
skeleton_viewer = SkeletonViewer(skeleton=skeleton, viewer=viewer, image=skeleton_image)

# set the size of the node points
skeleton_viewer.node_size = 3

In [13]:
skeleton_viewer.edges_layer.features

,start_node,end_node,validated,highlight
0,0,1,False,False
1,1,2,True,False
2,1,3,True,False
3,2,5,True,False
4,2,6,True,False


In [14]:
skeleton_viewer.nodes_layer.features

,node_index,contraction
0,0,NaN
1,1,NaN
2,2,"{4: {'node_coordinate': [40, 60, 30]}}"
3,3,NaN
4,5,NaN
5,6,NaN


## merging nodes with the viewer

you can also use the "merge nodes" widget to merge nodes by specifying the "node to keep" and "node to merge" in the UI and pressing the "merge nodes" button;